In [16]:
from data import *
from PIL import Image
import os, shutil

In [17]:
PATH = 'C:/Users/Kaare/Documents/Git projects/unet'

In [18]:
TRAIN_PATH = PATH + "/data/DRIVE/training"

TEST_PATH = PATH + "/data/DRIVE/test"

PROCESSED_TRAIN_PATH = PATH + '/data/DRIVE_preprocessed/train'

PROCESSED_TEST_PATH = PATH + '/data/DRIVE_preprocessed/test'

In [19]:
shutil.rmtree(PATH + "/data/DRIVE_preprocessed", ignore_errors=False, onerror=None)
os.mkdir(PATH + "/data/DRIVE_preprocessed")
os.mkdir(PROCESSED_TRAIN_PATH)
os.mkdir(PROCESSED_TEST_PATH)
os.mkdir(PROCESSED_TRAIN_PATH + "/images")
os.mkdir(PROCESSED_TRAIN_PATH + "/labels")
os.mkdir(PROCESSED_TEST_PATH + "/images")

In [20]:
Nimgs = len(os.listdir(TRAIN_PATH + '/images'))
channels = 1
height = 584
width = 565

i=1
for file in sorted(os.listdir(TRAIN_PATH + "/images")):
  img = cv2.imread(TRAIN_PATH + "/images/"+file, 0)
  img = np.expand_dims(img, axis=0)
  img = np.moveaxis(img,0, -1)
  #print(img.shape)
  cv2.imwrite(PROCESSED_TRAIN_PATH + "/images/" + str(i) + '.png',img)
  i += 1


i=1
for file in sorted(os.listdir(TRAIN_PATH + "/labels")):
  g_truth = Image.open(TRAIN_PATH + "/labels/" + file)
  cv2.imwrite(PROCESSED_TRAIN_PATH + "/labels/" + str(i) + '.png', np.asarray(g_truth))
  i += 1


In [21]:
Nimgs = len(os.listdir(TEST_PATH + '/images'))
channels = 1
height = 584
width = 565

i=1
for file in sorted(os.listdir(TEST_PATH + "/images")):
  img = cv2.imread(TEST_PATH + "/images/"+file, 0)
  img = np.expand_dims(img, axis=0)
  img = np.moveaxis(img,0, -1)
  #print(img.shape)
  cv2.imwrite(PROCESSED_TEST_PATH + "/images/" + str(i) + '.png',img)
  i += 1

In [22]:
pad(PROCESSED_TRAIN_PATH + "/images", PROCESSED_TRAIN_PATH + "/images", False)
pad(PROCESSED_TRAIN_PATH + "/labels", PROCESSED_TRAIN_PATH + "/labels", False)
pad(PROCESSED_TEST_PATH + "/images", PROCESSED_TEST_PATH + "/images", False)

Padding is done.
Padding is done.
Padding is done.


# data augmentation 

In deep learning tasks, a lot of data is need to train DNN model, when the dataset is not big enough, data augmentation should be applied.

keras.preprocessing.image.ImageDataGenerator is a data generator, which can feed the DNN with data like : (data,label), it can also do data augmentation at the same time.

It is very convenient for us to use keras.preprocessing.image.ImageDataGenerator to do data augmentation by implement image rotation, shift, rescale and so on... see [keras documentation](https://keras.io/preprocessing/image/) for detail.

For image segmentation tasks, the image and mask must be transformed **together!!**

## define your data generator

If you want to visualize your data augmentation result, set save_to_dir = your path

In [6]:
#if you don't want to do data augmentation, set data_gen_args as an empty dict.
data_gen_args = dict()

# data_gen_args = dict(rotation_range=0.2,
#                     width_shift_range=0.05,
#                     height_shift_range=0.05,
#                     shear_range=0.05,
#                     zoom_range=0.05,
#                     horizontal_flip=True,
#                     fill_mode='nearest')
myGenerator = trainGenerator(20,'data/membrane/train','image','label',data_gen_args,save_to_dir = "data/membrane/train/aug")

## visualize your data augmentation result

In [8]:
#you will see 60 transformed images and their masks in data/membrane/train/aug
num_batch = 3
for i,batch in enumerate(myGenerator):
    if(i >= num_batch):
        break

## create .npy data

If your computer has enough memory, you can create npy files containing all your images and masks, and feed your DNN with them.

In [9]:
image_arr,mask_arr = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#np.save("data/image_arr.npy",image_arr)
#np.save("data/mask_arr.npy",mask_arr)